In [40]:
import minari
import gymnasium as gym
from gato.tasks.control_task import ControlTask

In [39]:
import importlib
import gato.tasks.control_task
importlib.reload(gato.tasks.control_task)

<module 'gato.tasks.control_task' from '/home/eihli/src/NEKO/gato/tasks/control_task.py'>

In [7]:
dataset_name = 'BabyAI-GoToOpen-v0'
dataset = minari.load_dataset(dataset_name)

In [14]:
dataset, dataset.spec.env_spec.id

(<minari.dataset.minari_dataset.MinariDataset at 0x7fdb736327a0>,
 'BabyAI-GoToOpen-v0')

In [20]:
env = gym.make(dataset.spec.env_spec)

In [23]:
from argparse import Namespace

In [25]:
args = Namespace(patch_size=4)

In [22]:
context_len = 512
training_prompg_len_proportion=0.5
share_prompt_episodes=True
top_5_prompting=None

# Supported observation spaces

The first error I encounter is that the BabyAI environment has a Dict observation space.

What would it take to support that type of observation space?

Looking at how this is used, I see it conditionally:

- If it's a Box that has a shape of length 2 or 3
    - Adds image transforms
- Otherwise
    - Sets `obs_str` to `'continuous_obs'`, which eventually makes its way into `input_dict`, which eventually makes its way to `predict_control`, which doesn't get checked in `predict_control` but eventually makes its way to `tokenize_input_dicts`, and _that's_ where it gets checked.
        - It tokenizes the batch with the `continuous_obs_tokenizer`. 

(Is this a safe condition? Might there be non-image con)

## tokens_per_space

This is another place where the type of the observation space is checked.

Box: `space.shape[0]`
Discrete: `1`
Dict: ?

What is this?

`tokens_per_space` gets assigned to `action_tokens` [here](https://github.com/eihli/NEKO/blob/b66b48b88117307a442c43a7f4d8701706670144/gato/tasks/control_task.py#L74) and then used to calculate `tokens_per_timestamp` and is eventually used to create/manipulate the shape of the `input_dict` in [ControlTask.evaluate](https://github.com/eihli/NEKO/blob/b66b48b88117307a442c43a7f4d8701706670144/gato/tasks/control_task.py#L138).

In [34]:
env.observation_space

Dict('direction': Discrete(4), 'image': Box(0, 255, (7, 7, 3), uint8), 'mission': MissionSpace(<function BabyAIMissionSpace._gen_mission at 0x7fdb727b09d0>, None))

In [41]:
control = ControlTask('playground_env_name', env, dataset, context_len, args)

NotImplementedError: Unsupported space: Dict('direction': Discrete(4), 'image': Box(0, 255, (7, 7, 3), uint8), 'mission': MissionSpace(<function BabyAIMissionSpace._gen_mission at 0x7fdb727b09d0>, None))